### Task-1
## Decision Tree Classifier

In [1]:
import numpy as np

Let's start by implementing a node class for building the decision tree

In [2]:
class Node:
    def __init__ (self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left #feature<= threshold
        self.right = right #feature > threshold
        self.value = value

Let us now implement the `DecisionTreeClassifier` class

In [3]:
class DecisionTreeClassifier:
    """
    A simple implemetation of Decision Tree Classifier.

    parameters:
        function    : What method to use to calculate impurity. Currently supports "gini" and "entropy"
        max_depth   : Maximum allowed depth of tree
        min_samples : minimum number of samples going to a node for it to be a valid decision node.

    Methods (for user):
        fit()         : to learn the decision tree
        predict()     : predicts the label for a list of given points
        predict_one() : predicts for a single given point
        plot_tree()   : Plots the decision tree
    
    """
    def __init__(self, function="gini", max_depth=None, min_samples = None):

        #checking if valid function provided or not
        if(function not in ["gini", "entropy"]):
            raise ValueError('The parameter "function" can only take values "gini" or "entropy".')

        #checking if valid max_depth provided or not
        if(max_depth != None and (max_depth==0 or not max_depth.is_integer())):
            raise ValueError("The parameter max_depth needs to be an integer greater than or equal to 1")

        self.function=function
        self.root = None
        self.max_depth = max_depth
        self.min_samples = min_samples

Function to calculate the impurity. 

$$\text{Gini Impurity} = 1 - \sum_{i=1}^n p_i^2$$

$$\text{Entropy} = \sum_{i=1}^n -p_i \log(p_i)$$

Where $p_i$ are the probability of the $n$ different types of classes present in that node.

In [4]:
def calculate_impurity(self, y):
    impurity = None

    #finding the distinct values in y
    distinct = set(y)
    probabilities = []

    #calculating probabilities
    for element in distinct:
        freq = np.sum(y==element)
        probabilities.append(freq/len(y))

    #calculating impurity according to provided function
    if(self.function=="gini"):
        impurity = 1
        for prob in probabilities:
            impurity -= prob**2
    else:
        impurity = 0
        for prob in probabilities:
            if(prob>0):
                impurity += (-1) * prob * np.log2(prob)

    return impurity

DecisionTreeClassifier.calculate_impurity = calculate_impurity

This finds the best feature and threshold to split. It goes through every feature with every possible value of threshold and returns the one with minimum weighted children Impurity, where
$$\text{Weighted children Entropy} = p(\text{left node}) \cdot \text{left node impurity} + p(\text{right node}) \cdot \text{right node impurity}$$
The probability of a child node is calculated as follows
$$p(\text{child node}) = \frac{\text{samples going to this child node}}{\text{total samples in current node}}$$

In [5]:
def find_best_split(self, X,y):
    optimal_entropy = np.inf #max can be inf
    optimal_feature = None

    #going through each feature
    for index in range(X.shape[1]):
        #testing each possible split
        #first will get the splits by getting the unique elements
        unique_elements = set(X[:, index])
        unique_elements = list(unique_elements)
        unique_elements.sort()

        for i in range(len(unique_elements) - 1):
            temp_threshold = (unique_elements[i] + unique_elements[i+1])/2

            left_values = []
            right_values = []

            for j in range(len(X)):
                if(X[j,index] <= temp_threshold):
                    left_values.append(j)
                else:
                    right_values.append(j)



            left_impurity = self.calculate_impurity([y[index] for index in left_values])
            right_impurity = self.calculate_impurity([y[index] for index in right_values])

            weighted_entropy = (len(left_values) / len(X)) * left_impurity + (len(right_values) / len(X)) * right_impurity

            #checking if we got better gain in this case
            if(optimal_entropy >= weighted_entropy):
                optimal_entropy = weighted_entropy
                optimal_feature = (index, temp_threshold)

    #by now we have our optimal feature to split across
    return optimal_feature[0], optimal_feature[1]

DecisionTreeClassifier.find_best_split = find_best_split

fit function, that learns the tree

In [6]:
def fit(self, X, y):
    #checking if they have the same size
    if(len(X)!=len(y)):
        raise ValueError("The lengths of X and y do not match.")

    #start building the tree
    self.root = self.build_tree(X,y,0)

DecisionTreeClassifier.fit = fit

Main learning function. First checks if the this Node can be a decision node or not. if not, returns as a leaf node. If it can become a decision node, finds the optimal feature to split on. It splits according to that and calls itself again to learn further from the children Nodes. finally returns a decision node with the feature, threshold and children nodes.

In [7]:
def build_tree(self, X, y, depth):

    #taking out all available labels in an array
    labels = np.unique(y)

    #checking if all labels are same, then return a leaf Node
    if(len(labels)==1):
        return Node(value=y[0])

    #checking if more than max_depth
    if(self.max_depth != None and self.max_depth <= max_depth):
        count = np.array([sum(y==label) for label in labels])
        return Node(value=labels[np.argmax(count)])

    #checking if number of samples has fallen below a threshold
    if(self.min_samples != None and len(X)<min_samples):
        count = np.array([sum(y==label) for label in labels])
        return Node(value=labels[np.argmax(count)])

    #we can split
    #lets find the best features and threshold to split
    feature_index, threshold = self.find_best_split(X,y)


    #splitting the data accordingly
    left_X =[]
    left_y =[]
    right_X = []
    right_y = []
    for i in range(len(X)):
        if (X[i, feature_index] <= threshold):
            left_X.append(X[i])
            left_y.append(y[i])
        else:
            right_X.append(X[i])
            right_y.append(y[i])


    left_X = np.array(left_X)
    left_y = np.array(left_y)
    right_X = np.array(right_X)
    right_y = np.array(right_y)

    
    #making the tree for each of the children of this Node by recusrion
    left_node = self.build_tree(left_X, left_y, depth+1)
    right_node = self.build_tree(right_X, right_y, depth+1)


    #return the built tree (or subtree)
    return Node(feature_index = feature_index, threshold = threshold, left=left_node, right= right_node)

DecisionTreeClassifier.build_tree = build_tree

This function predicts the label for a new data point. It travels the tree until it reaches a leaf node and returns the leaf node value.

In [8]:
def predict(self, X_test, labels):
    predicted_labels=[]
    for x in X_test:
        predicted_labels.append(labels[self.predict_helper(x, self.root)])
    return predicted_labels


def predict_one(self, x, labels):
    return labels[self.traverse_tree(x, self.root)]


def predict_helper(self, x, current_node):

    #checking if current node is a leaf or not
    if(current_node.value != None):
        return current_node.value

    #current node is not leaf node
    index = current_node.feature_index
    threshold = current_node.threshold

    if(x[index] <= threshold):
        return self.predict_helper(x,current_node.left)
    else:
        return self.predict_helper(x,current_node.right)

DecisionTreeClassifier.predict = predict
DecisionTreeClassifier.predict_one = predict_one
DecisionTreeClassifier.predict_helper = predict_helper

Now that we are done with the implementation, let's put it into action.

In [9]:
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

decision_tree_classifer = DecisionTreeClassifier(function="entropy")

X, labels_train = np.array([row[:-1] for row in data]), np.array([row[-1] for row in data])

#encoding the y labels
#Wine = 0, Beer = 1, Whiskey = 2
y=[]
for i in range(len(labels_train)):
    if(labels_train[i]=="Wine"):
        y.append(0)
    elif(labels_train[i]=="Beer"):
        y.append(1)
    else:
        y.append(2)

labels =["Wine","Beer", "Whiskey"]


decision_tree_classifer.fit(X,y)

#checking if it correctly predicts the training data
decision_tree_classifer.predict(X, labels) == labels_train

array([ True,  True,  True,  True,  True,  True,  True,  True])

Great, it predicts all the training points correctly (it should though). \
Lets try to test it on the `test_data` given.

In [10]:
test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

y_predicted = decision_tree_classifer.predict(test_data, labels)
print(y_predicted)

['Beer', 'Whiskey', 'Wine']


Hurray! It works for the test data as well. \
Implemnting the Plot tree function for printing the decision 
tree

In [11]:
def plot_tree(self, feature_names, labels):
    self.traverse_print(self.root, feature_names,labels, append="")

def traverse_print(self, node, feature_names,labels ,append=""):
    if(node.value!=None):#Leaf node
        print(append + f"It is {labels[node.value]}.")
        return

    print(append  + f"├─If {feature_names[node.feature_index]} <= {node.threshold} : ")
    self.traverse_print(node.left, feature_names,labels, append+"│  ")
    print(append  + f"└─Else ({feature_names[node.feature_index]} > {node.threshold}): ")
    self.traverse_print(node.right, feature_names, labels, append+"  ")


DecisionTreeClassifier.plot_tree = plot_tree
DecisionTreeClassifier.traverse_print = traverse_print

Lets test that out and have a look at our decision Tree.

In [12]:
decision_tree_classifer.plot_tree(["Alcohol Content", "Sugar", "Color"], ["Wine", "Beer", "Whiskey"])

├─If Color <= 0.5 : 
│  It is Beer.
└─Else (Color > 0.5): 
  ├─If Sugar <= 0.65 : 
  │  It is Whiskey.
  └─Else (Sugar > 0.65): 
    It is Wine.


We can clearly see our decision tree. It doesn't consider alcohol content for now because we have a small number of data points. As the number of data points increase, it will start considering alochol content also for deciding (if it affects the decision).